**Classification using Neural Network:**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("customer_churn.csv")
df.head()

In [ ]:
df.info()

**A) Data Manipulation:**

In [ ]:
#remove invalid records and convert object to float
df = df[df['TotalCharges']!= ' ']  #11 records removed
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'])

In [ ]:
#a. Find the total number of male customers
len(df[df['gender']=='Male'])

In [ ]:
#b. Find the total number of customers whose Internet Service is ‘DSL’
len(df[df['InternetService']=='DSL'])

In [ ]:
#c. Extract all the Female senior citizens whose Payment Method is Mailed check & store the
# result in ‘new_customer’
new_customer = df[(df['gender']=='Female') & (df['SeniorCitizen']==1) & (df['PaymentMethod']=='Mailed check')]
new_customer

In [ ]:
#d. Extract all those customers whose tenure is less than 10 months or their Total charges is less
# than 500$ & store the result in ‘new_customer’
new_customer = df[(df['tenure']<10) | (df['TotalCharges']<500)]
new_customer

**B) Data Visualization:**

In [ ]:
#convert target field from object to int
df['Churn'] = df['Churn'].replace(['Yes', 'No'], [1, 0])

In [ ]:
#a. Build a pie-chart to show the distribution of customers would be churning out
vc = df['Churn'].value_counts()
plt.pie(vc, labels=["Not Churned","Churned"], autopct='%1.0f%%')
plt.show()

In [ ]:
#b. Build a bar-plot to show the distribution of ‘Internet Service’
vc = df['InternetService'].value_counts()
plt.bar(vc.index, vc.values)
plt.xlabel('Internet Service')
plt.ylabel('Number of Customers')
plt.title('Distribution of Internet Service')
plt.show()

**C) Model Building:**

**a.** Build a sequential model using Keras, to find out if the customer would churn or not

In [ ]:
X = df['tenure']
y = df['Churn']

In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [ ]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X,y, test_size=0.2, random_state=33)
Xtest.shape, ytest.shape

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Input(shape=(1,)))
model.add(tf.keras.layers.Dense(8, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(), metrics=['accuracy'])
history = model.fit(Xtrain, ytrain, epochs=150, batch_size=32)

In [ ]:
#predict and plot confusion matrix
from sklearn.metrics import confusion_matrix
from seaborn import heatmap

def plot_CM(xtest, ytest):
  ypred = model.predict(xtest)
  ypred = np.round(ypred).flatten()
  cm = confusion_matrix(ytest, ypred)
  heatmap(cm, annot=True, fmt='d', cmap='Blues')

In [ ]:
plot_CM(Xtest, ytest)
#model.evaluate(Xtest,ytest)

In [ ]:
# Plot training accuracy over epochs
def plot_history(history):
  plt.plot(history.history['accuracy'])
  plt.title('Model Accuracy')
  plt.xlabel('Epoch')
  plt.ylabel('Accuracy')
  plt.show()

In [ ]:
plot_history(history)

**b.** Build the 2nd model using same target and feature variables: (use dropouts)

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Input(shape=(1,)))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(8, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(), metrics=['accuracy'])
history = model.fit(Xtrain, ytrain, epochs=150, batch_size=32)

In [ ]:
plot_CM(Xtest, ytest)

In [ ]:
plot_history(history)

**c.** Build the 3rd model using ‘Tenure’, ’Monthly Charges’ & ‘Total Charges’ as the features and
‘Churn’ as the dependent/target column:

In [ ]:
X = df[['tenure','MonthlyCharges','TotalCharges']]
Xtrain, Xtest, ytrain, ytest = train_test_split(X,y, test_size=0.2, random_state=33)
Xtest.shape, ytest.shape

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Input(shape=(3,)))
model.add(tf.keras.layers.Dense(8, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(), metrics=['accuracy'])
history = model.fit(Xtrain, ytrain, epochs=150, batch_size=32)

In [ ]:
plot_CM(Xtest, ytest)

In [ ]:
plot_history(history)